In [93]:
### imports 
import matplotlib.pyplot as plt 
import pickle
import regex
from bs4 import BeautifulSoup
import xml.etree.ElementTree as etree
from pathlib import Path
import os
import requests

### Sort bills by size

```sh
mbp-filip:NLP filip$ ls -lSh ustawy_cleaned/ | head -50
total 71584
-rw-r--r--  1 filip  staff   566K Nov 26 16:46 2000_696.txt
-rw-r--r--  1 filip  staff   411K Nov 26 16:46 1996_465.txt
-rw-r--r--  1 filip  staff   404K Nov 26 16:46 2001_627.txt
-rw-r--r--  1 filip  staff   387K Nov 26 16:46 1997_555.txt
-rw-r--r--  1 filip  staff   344K Nov 26 16:46 2002_1689.txt
-rw-r--r--  1 filip  staff   269K Nov 26 16:46 2000_1186.txt
-rw-r--r--  1 filip  staff   264K Nov 26 16:46 1998_1118.txt
-rw-r--r--  1 filip  staff   257K Nov 26 16:46 1997_117.txt
-rw-r--r--  1 filip  staff   253K Nov 26 16:46 2001_1070.txt
-rw-r--r--  1 filip  staff   252K Nov 26 16:46 2001_1368.txt
-rw-r--r--  1 filip  staff   246K Nov 26 16:46 2001_499.txt
-rw-r--r--  1 filip  staff   242K Nov 26 16:46 2003_1750.txt
-rw-r--r--  1 filip  staff   239K Nov 26 16:46 2001_1545.txt
-rw-r--r--@ 1 filip  staff   238K Nov 26 16:46 2000_991.txt
-rw-r--r--  1 filip  staff   235K Nov 26 16:46 1997_714.txt
-rw-r--r--  1 filip  staff   233K Nov 26 16:46 2001_1229.txt
-rw-r--r--  1 filip  staff   229K Nov 26 16:46 2000_1268.txt
-rw-r--r--  1 filip  staff   228K Nov 26 16:46 1994_195.txt
-rw-r--r--  1 filip  staff   228K Nov 26 16:46 2003_2277.txt
-rw-r--r--  1 filip  staff   228K Nov 26 16:46 1997_926.txt
-rw-r--r--  1 filip  staff   225K Nov 26 16:46 2000_1104.txt
-rw-r--r--  1 filip  staff   221K Nov 26 16:46 1997_553.txt
-rw-r--r--  1 filip  staff   214K Nov 26 16:46 2004_2065.txt
-rw-r--r--@ 1 filip  staff   213K Nov 26 16:46 2004_1693.txt
-rw-r--r--  1 filip  staff   211K Nov 26 16:46 2004_880.txt
-rw-r--r--  1 filip  staff   207K Nov 26 16:46 2004_177.txt
-rw-r--r--  1 filip  staff   186K Nov 26 16:46 2003_423.txt
-rw-r--r--@ 1 filip  staff   184K Nov 26 16:46 1997_349.txt
-rw-r--r--  1 filip  staff   179K Nov 26 16:46 2000_1315.txt
-rw-r--r--  1 filip  staff   178K Nov 26 16:46 1999_930.txt
-rw-r--r--  1 filip  staff   178K Nov 26 16:46 1994_591.txt
-rw-r--r--  1 filip  staff   176K Nov 26 16:46 2004_2533.txt
-rw-r--r--  1 filip  staff   176K Nov 26 16:46 1996_110.txt
-rw-r--r--  1 filip  staff   175K Nov 26 16:46 1997_557.txt
-rw-r--r--  1 filip  staff   170K Nov 26 16:46 2001_1381.txt
-rw-r--r--  1 filip  staff   166K Nov 26 16:46 1999_95.txt
-rw-r--r--  1 filip  staff   164K Nov 26 16:46 2000_136.txt
-rw-r--r--  1 filip  staff   159K Nov 26 16:46 1996_460.txt
-rw-r--r--  1 filip  staff   157K Nov 26 16:46 2001_1438.txt
-rw-r--r--  1 filip  staff   151K Nov 26 16:46 1996_561.txt
-rw-r--r--  1 filip  staff   149K Nov 26 16:46 2001_1188.txt
-rw-r--r--@ 1 filip  staff   147K Nov 26 16:46 2001_628.txt
-rw-r--r--  1 filip  staff   146K Nov 26 16:46 2001_475.txt
-rw-r--r--  1 filip  staff   145K Nov 26 16:46 2003_2256.txt
-rw-r--r--  1 filip  staff   145K Nov 26 16:46 1998_602.txt
-rw-r--r--  1 filip  staff   140K Nov 26 16:46 2001_906.txt
-rw-r--r--@ 1 filip  staff   139K Nov 26 16:46 2001_92.txt
-rw-r--r--  1 filip  staff   138K Nov 26 16:46 1996_496.txt
-rw-r--r--  1 filip  staff   133K Nov 26 16:46 1997_153.txt
```

### Clean bills from eventual html, but leave capitalized letters (in comparison to ex. 5)

In [2]:
acts_directory = Path("/Users/filip/Documents/II_semestr/NLP/ustawy/")
files = acts_directory.glob('*')
os.makedirs("/Users/filip/Documents/II_semestr/NLP/ustawy_cleaned_capitalized/", exist_ok=True)
acts_cleaned_dir = Path("/Users/filip/Documents/II_semestr/NLP/ustawy_cleaned_capitalized/")

for file in files: 
    file_name = file.name
    with open(file, 'r', encoding='utf-8') as f:
        cnt = regex.sub("\p{Cf}", "", f.read())
        cnt = cnt.replace("-","")
        #cnt2 = regex.sub("\p{Z}+", " ", cnt)
        act_cleaned = BeautifulSoup(cnt, "lxml").get_text()  ## remove all html code

    cleaned_file_path = acts_cleaned_dir.joinpath(file_name)
    with open(cleaned_file_path, "w", encoding="utf-8") as f:
        f.write(act_cleaned)

In [3]:
biggest_bills = ["2000_696.txt","1996_465.txt","2001_627.txt","1997_555.txt",
                 "2002_1689.txt","2000_1186.txt","1998_1118.txt","1997_117.txt","2001_1070.txt","2001_1368.txt",
                 "2001_499.txt","2003_1750.txt","2001_1545.txt","2000_991.txt","1997_714.txt","2001_1229.txt",
                 "2000_1268.txt","1994_195.txt","2003_2277.txt","1997_926.txt","2000_1104.txt","1997_553.txt",
                 "2004_2065.txt","2004_1693.txt","2004_880.txt","2004_177.txt","2003_423.txt","1997_349.txt",
                 "2000_1315.txt","1999_930.txt","1994_591.txt","2004_2533.txt","1996_110.txt","1997_557.txt",
                 "2001_1381.txt","1999_95.txt","2000_136.txt","1996_460.txt","2001_1438.txt","1996_561.txt",
                 "2001_1188.txt","2001_628.txt","2001_475.txt","2003_2256.txt","1998_602.txt","2001_906.txt",
                 "2001_92.txt","1996_496.txt","1997_153.txt"]

In [4]:
pref = "/Users/filip/Documents/II_semestr/NLP/ustawy_cleaned_capitalized/"
biggest_bills_pwd = list(map(lambda p: f"{pref}{p}", biggest_bills))

In [ ]:
biggest_bills_pwd

### ReTag everything as in ex. 5

In [6]:
## takie podejście usuwa też zdublowane tokeny 
tokens_patt = regex.compile("(?:\\n|^).+?\\t")  #matches polish characters as well
def tokens_analysis_dct(text):
    words_analysis = dict()

    previous_word = None
    bigrams = []
    
    for match in regex.finditer(tokens_patt, text):

        if r"\n\t" not in repr(match[0]):
            if not previous_word:
                previous_word = match[0].strip()
                prev_word_end = match.end()
            else:
                bigrams.append((previous_word, match[0].strip()))
                words_analysis[previous_word] = list(map(str.strip,text[prev_word_end:match.start()].split("\t")))
                previous_word = match[0].strip()
                prev_word_end = match.end()
    words_analysis["bigrams"] = bigrams
    return words_analysis

In [7]:
files_analyzed = dict()
counter = 0 
s = requests.Session()
for file in biggest_bills_pwd:
    counter += 1
    if counter % 10 == 0:
        print(counter)
    with open(file, "r", encoding='utf-8') as f:
        body = f.read().encode(encoding='utf-8')
        res = s.post("http://localhost:9200",data=body)
        words_analysis = tokens_analysis_dct(res.text)
        files_analyzed[f.name] = words_analysis

10
20
30
40


In [11]:
with open("file_analysed_capitalised", "wb") as f:
    pickle.dump(files_analyzed, f)

### Consecutive capital words expressions from ex. 5 (lemmatized and sentece split documents) 

In [14]:
files_analyzed[biggest_bills_pwd[0]]["bigrams"]

[('Dz', '.'),
 ('.', 'U'),
 ('U', '.'),
 ('.', 'z'),
 ('z', '1998'),
 ('1998', 'r'),
 ('r', '.'),
 ('.', 'Nr'),
 ('Nr', '106'),
 ('106', ','),
 (',', 'poz'),
 ('poz', '.'),
 ('.', '668'),
 ('668', 'USTAWA'),
 ('USTAWA', 'z'),
 ('z', 'dnia'),
 ('dnia', '24'),
 ('24', 'lipca'),
 ('lipca', '1998'),
 ('1998', 'r'),
 ('r', '.'),
 ('.', 'o'),
 ('o', 'zmianie'),
 ('zmianie', 'niektórych'),
 ('niektórych', 'ustaw'),
 ('ustaw', 'określających'),
 ('określających', 'kompetencje'),
 ('kompetencje', 'organów'),
 ('organów', 'administracji'),
 ('administracji', 'publicznej'),
 ('publicznej', 'w'),
 ('w', 'związku'),
 ('związku', 'z'),
 ('z', 'reformą'),
 ('reformą', 'ustrojową'),
 ('ustrojową', 'państwa'),
 ('państwa', 'Art'),
 ('Art', '.'),
 ('.', '1'),
 ('1', '.'),
 ('.', 'W'),
 ('W', 'ustawie'),
 ('ustawie', 'z'),
 ('z', 'dnia'),
 ('dnia', '24'),
 ('24', 'marca'),
 ('marca', '1920'),
 ('1920', 'r'),
 ('r', '.'),
 ('.', 'o'),
 ('o', 'nabywaniu'),
 ('nabywaniu', 'nieruchomości'),
 ('nieruchomości'

In [ ]:
## workflow 
usuwam bigramy, gdzie bigram[0] == "."
zbieram znalezione w taki sposób entities do listy  -- dla każdego pliku w taki sposób mam zdublowane entities w różnych odmianach
zamieniam entities na entites lemmatized

In [84]:
def find_entities(krrnt_dct):
    entities_tmp = []
    entities = dict()
    for ff, dct in krrnt_dct.items():
        for bg in dct["bigrams"]:
            if bg[0] == "." and bg[1][0].isupper() or bg[0] != "." and bg[1][0].isupper():
                entities_tmp.append(bg[1])
            else:
                if entities_tmp and len(entities_tmp) > 1:
                    try:
                        entities[ff].append(tuple(entities_tmp))
                    except KeyError:
                        entities[ff] = [tuple(entities_tmp)]
                    entities_tmp.clear()
                else: # i want to find only consecutive entities CAN BE SWITCHED OFF 
                    entities_tmp.clear()
        # clear na koniec, jeśli zostaje jakieś entity na końcu pliku
        if entities_tmp:
            try:
                entities[ff].append(tuple(entities_tmp))
            except KeyError:
                entities[ff] = [tuple(entities_tmp)]
                entities_tmp.clear()
    return entities

In [ ]:
files_analyzed

In [85]:
entites_raw = find_entities(files_analyzed)

In [86]:
def get_lemmatized_entities_single(entities):
    def lemmatize_entities(ent: tuple):
        #files_analyzed is global 
        bf = []
        lematized = files_analyzed[ff]
        for e in ent:
            bf.append(lematized[e][1])  #todo
        return tuple(bf)
    # files_analyzed is global
    entities_lematized = dict()
    for ff, ents in entities.items():
        entities_lematized[ff] = set(list(map(lemmatize_entities, ents)))
        
    return entities_lematized

In [87]:
entities_lematized = get_lemmatized_entities_single(entites_raw)

In [88]:
entities_lematized[biggest_bills_pwd[0]]

{('MinisterSzef', 'urząd', 'rada', 'minister'),
 ('Polska', 'związek', 'działkowiec'),
 ('Rzeczpospolita', 'polski'),
 ('Warszawa', 'centrum'),
 ('Ziemia', 'odzyskać'),
 ('agencja', 'restrukturyzacja'),
 ('biblioteka', 'narodowy'),
 ('centralny', 'fundusz', 'gospodarka', 'zasób', 'geodezyjny'),
 ('centralny', 'muzeum', 'pożarnictwo'),
 ('centralny', 'rada', 'związek', 'zawodowy'),
 ('centralny', 'rejestr', 'skazany'),
 ('członek', 'naczelny', 'rada', 'zatrudnienie'),
 ('do', 'generalny', 'dyrektor', 'droga', 'publiczny'),
 ('dyrektor', 'centralny', 'inspektorat'),
 ('dyrektor', 'generalny'),
 ('dziennik', 'ustawa', 'Rzeczpospolita', 'polski'),
 ('eksploatacja', 'autostrada'),
 ('etat', 'policja'),
 ('federacja', 'Rosyjski'),
 ('fundusz', 'gospodarka', 'zasób', 'geodezyjny'),
 ('fundusz', 'praca'),
 ('generalny', 'Konserwator', 'zabytek'),
 ('generalny', 'dyrekcja', 'droga', 'publiczny'),
 ('generalny', 'dyrektor', 'droga', 'publiczny'),
 ('generalny', 'konserwator', 'zabytek'),
 ('gosp

In [89]:
##  zliczanie wystąpięń poszczególnych entities globalnie w 50 ustawach
global_entities_count = dict()
for _,entities in entities_lematized.items():
    for ent in entities:
        try:
            global_entities_count[ent] +=1
        except KeyError:
            global_entities_count[ent] = 1

In [90]:
global_entities_count_srted = sorted(global_entities_count.items(), key=lambda it: it[1], reverse=True)

In [91]:
global_entities_count_srted[:50]

[(('Rzeczpospolita', 'polski'), 44),
 (('skarb', 'państwo'), 38),
 (('rada', 'minister'), 36),
 (('prezes', 'rada', 'minister'), 28),
 (('minister', 'obrona', 'narodowy'), 26),
 (('dziennik', 'urzędowy', 'Rzeczpospolita', 'polski'), 26),
 (('monitor', 'polski'), 24),
 (('minister', 'sprawiedliwość'), 19),
 (('unia', 'europejski'), 19),
 (('straż', 'graniczny'), 19),
 (('prezes', 'główny', 'urząd', 'statystyczny'), 17),
 (('Rzeczpospolita', 'Polska'), 16),
 (('zakład', 'ubezpieczenie', 'społeczny'), 15),
 (('służba', 'więzienny'), 15),
 (('urząd', 'ochrona', 'państwo'), 15),
 (('minister', 'finanse'), 14),
 (('naczelny', 'sąd', 'administracyjny'), 14),
 (('państwowy', 'straż', 'pożarny'), 13),
 (('prezydent', 'Rzeczpospolita', 'polski'), 13),
 (('siła', 'zbrojny', 'Rzeczpospolita', 'polski'), 12),
 (('marszałek', 'sejm'), 12),
 (('narodowy', 'bank', 'polski'), 12),
 (('szef', 'urząd', 'ochrona', 'państwo'), 11),
 (('dziennik', 'ustawa', 'Rzeczpospolita', 'polski'), 10),
 (('komisja', 'e

### NER algorithm applied

In [8]:
coarse_grained_classes = ["nam_loc", "nam_fac", "nam_liv", "nam_org_nation",
                         "nam_org", "nam_eve", "nam_pro", "nam_adj",
                         "nam_num", "nam_oth"]

In [167]:
files_nered = Path("/Users/filip/Documents/II_semestr/NLP/lab8_ner/out1").glob("*")

cc = 0
entities_nered_tmp = dict()
entities_nered = dict()
for file in files_nered:
    xml_iter = etree.iterparse(file)
    for _,elem in xml_iter:
        if elem.tag == "tok":
            children = elem.findall("./*")
            for child in children:
                if child.tag == "orth": name = child.text
                if child.tag == "lex": 
                    base = child.find("base").text                    
                if child.tag == "ann" and "head" in child.attrib: # zaczynam trackować czanel,numer
                    entities_nered_tmp[(child.attrib["chan"], child.text)] = [(name, base)]
                #there is no head, but still track channel
                elif child.tag == "ann" and child.text != "0" and (child.attrib["chan"], child.text) not in entities_nered_tmp:
                    entities_nered_tmp[(child.attrib["chan"], child.text)] = [(name, base)]
                elif child.tag == "ann" and "chan" in child.attrib and child.text != "0":
                    entities_nered_tmp[(child.attrib["chan"],child.text)].append((name, base))
                    
        if elem.tag == "sentence": # end of sentence
            for ch, val in entities_nered_tmp.items():
                try:
                    entities_nered[ch[0]].append(tuple(val))
                except KeyError:
                    entities_nered[ch[0]] = [tuple(val)]
            entities_nered_tmp.clear()


In [168]:
len(entities_nered.keys())

50

In [172]:
for i in entities_nered.keys():
    print(i)

nam_pro_media_periodic
nam_adj_country
nam_org_institution
nam_loc_gpe_country
nam_org_organization
nam_pro_title_document
nam_org_company
nam_pro_brand
nam_liv_person
nam_pro_title
nam_oth_currency
nam_loc_gpe_city
nam_oth_position
nam_loc_astronomical
nam_eve_human
nam_fac_goe
nam_org_group_team
nam_oth_tech
nam_org_political_party
nam_loc_other
nam_loc_gpe_admin1
nam_adj_other
nam_pro_media_web
nam_loc_gpe_admin2
nam_pro_software
nam_eve_human_sport
nam_pro_other
nam_fac_system
nam_fac_road
nam_pro_title_book
nam_pro_model_car
nam_loc_country_region
nam_loc_gpe_admin3
nam_adj_city
nam_loc_historical_region
nam_org_nation
nam_loc_land_island
nam_eve_other
nam_liv_god
nam_num_house
nam_oth_other
nam_org_group_band
nam_loc_land_continent
nam_loc_gpe_district
nam_num_phone
nam_loc_hydronym_river
nam_fac_square
nam_loc_hydronym_sea
nam_pro_media_tv
nam_loc_gpe_subdivision


In [139]:
## leave out only entities that have more than 2 members
entities_nered_map = {k:list(filter(lambda v: len(v) > 1,val)) for k,val in entities_nered.items()}

In [138]:
entities_nered_map

{'nam_pro_media_periodic': [(('Dz', 'Dzieje_(Apostolskie)'),
   ('.', '.'),
   ('U', 'u'),
   ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolski

In [114]:
list(entities_nered_map)

[[(('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'dziennik'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'dziennik'), ('.', '.')),
  (('Dz', 'dziennik'),),
  (('Dz', 'dziennik'),),
  (('Dz', 'dziennik'),),
  (('Dz', 'dziennik'),),
  (('Dz', 'dziennik'),),
  (('Dz', 'dziennik'), ('.', '.')),
  (('Dz', 'dziennik'), ('.', '.')),
  (('Dz', 'dziennik'),),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.')),
  (('Dz', 'Dzieje_(Apostolskie)'), ('.', '.'), ('U', 'u'), ('.', '.'))],
 [(('polski', 'polski'),),
  (('p

### frequency of the corse grained classes 

In [178]:
counter = dict()
for class_ in coarse_grained_classes:
    for key in entities_nered.keys():
        if class_ in key:
            try:
                counter[class_] += entities_nered[key] 
            except KeyError:
                counter[class_] = [entities_nered[key]]

for c, v in counter.items():
    print(f"class {c}: {len(v)}")

class nam_loc: 471
class nam_fac: 8
class nam_liv: 50
class nam_org_nation: 1
class nam_org: 2059
class nam_eve: 12
class nam_pro: 1501
class nam_adj: 78
class nam_num: 2
class nam_oth: 82


In [181]:
## most frequent entities for each coarse class

ents_freq = dict()
for cl in coarse_grained_classes:
    ents_freq[cl] = dict()
    for v in counter[cl][0]:
        try:
            ents_freq[cl][v] += 1
        except KeyError:
            ents_freq[cl][v] = 1
    

In [182]:
ents_freq

{'nam_loc': {(('Rzeczpospolitą', 'rzeczpospolita'), ('Polską', 'Polska')): 39,
  (('Rzeczypospolitej', 'rzeczpospolita'), ('Polskiej', 'polski')): 812,
  (('Polski', 'Polska'),): 9,
  (('Polsce', 'Polska'),): 51,
  (('Polska', 'Polska'),): 2,
  (('Rzeczpospolita', 'rzeczpospolita'), ('Polska', 'Polska')): 52,
  (('Polską', 'Polska'),): 10,
  (('Rzecząpospolitą', 'rzeczpospolita'), ('Polską', 'Polska')): 3,
  (('Konfederacją', 'konfederacja'), ('Szwajcarską', 'szwajcarski')): 1,
  (('Konfederacji', 'konfederacja'), ('Szwajcarskiej', 'szwajcarski')): 11,
  (('Konfederację', 'konfederacja'), ('Szwajcarską', 'szwajcarski')): 2,
  (('Rzeczypospolitej', 'rzeczpospolita'), ('Polski', 'Polska')): 1,
  (('Królestwem', 'królestwo'), ('Belgii', 'Belgia')): 1,
  (('Królestwem', 'królestwo'), ('Danii', 'Dania')): 1,
  (('Republiką', 'republika'),
   ('Federalną', 'federalny'),
   ('Niemiec', 'Niemcy')): 1,
  (('Republiką', 'republika'), ('Grecką', 'grecki')): 1,
  (('Królestwem', 'królestwo'), ('Hi

In [ ]:
global_entities_count_srted = sorted(global_entities_count.items(), key=lambda it: it[1], reverse=True)

In [186]:
for cl in ents_freq.keys():
    ents_cl_ent_sorted = sorted(ents_freq[cl].items(), key=lambda it: it[1], reverse=True)
    print(f"{cl}")
    for i in ents_cl_ent_sorted[:10]:
          print(i)
    print("=====")

nam_loc
((('Rzeczypospolitej', 'rzeczpospolita'), ('Polskiej', 'polski')), 812)
((('Rzeczpospolita', 'rzeczpospolita'), ('Polska', 'Polska')), 52)
((('Polsce', 'Polska'),), 51)
((('Rzeczpospolitą', 'rzeczpospolita'), ('Polską', 'Polska')), 39)
((('Konfederacji', 'konfederacja'), ('Szwajcarskiej', 'szwajcarski')), 11)
((('Polską', 'Polska'),), 10)
((('Polski', 'Polska'),), 9)
((('Rzecząpospolitą', 'rzeczpospolita'), ('Polską', 'Polska')), 3)
((('Republiki', 'republika'), ('Cypryjskiej', 'cypryjski')), 3)
((('Rzeczpospolitej', 'rzeczpospolita'), ('Polskiej', 'polski')), 3)
=====
nam_fac
((('Inspektor', 'inspektor'), ('Nadzoru', 'nadzór'), ('Wewnętrznego', 'wewnętrzny')), 49)
((('Komendant', 'komendant'), ('Główny', 'główny')), 16)
((('Straży', 'straża'), ('Granicznej', 'graniczny')), 11)
((('Mangan', 'mangan'),), 8)
((('Obrony', 'obrona'), ('Narodowej', 'narodowy')), 6)
((('Krajowego', 'krajowy'), ('Rejestru', 'rejestr'), ('Karnego', 'karny')), 5)
((('Zasłużonego', 'zasłużony'), ('Dawcy'

In [ ]:
## Pytania
